In [2]:
import numpy as np
import matplotlib.pyplot as plt

import quantumsolver as qs

#help(qs.fewnucleonEmulator)

# Vectorized version of the fewnucleonEmulator
vfewnucleonEmulator = np.vectorize(qs.fewnucleonEmulator)

# Test of vectorized version
cD_grid = np.array([-2.5, 2.5])
cE_grid = np.array([-1., 1.])
# Create a 2-dimensional grid
cD_eval, cE_eval = np.meshgrid(cD_grid,cE_grid)
print('cD_eval is of shape:', cD_eval.shape)
print(cD_eval)
# The vectorized version of the few-nucleon emulator will produce an output tuple
# where each element is an array of the same shape as the input
E4He_eval, Rp4He_eval, Rch4He_eval, E3He_eval, E3H_eval, E1A3H_eval, fT3H_eval = vfewnucleonEmulator(cD_eval, cE_eval)
print('E4He_eval is of shape:', E4He_eval.shape)
print(E4He_eval)

cD_eval is of shape: (2, 2)
[[-2.5  2.5]
 [-2.5  2.5]]
E4He_eval is of shape: (2, 2)
[[ -24.40247682  -21.37114724]
 [-105.55765698  -40.33178611]]
